In [6]:
# Dependencies
import requests
import pandas as pd
from census import Census
import hvplot.pandas
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt

In [4]:
# Import U.S. Census API Key
from config import api_key

# Create an instance of the Census library
c = Census(api_key)

In [5]:
# Specify the variables 
# B08301_001E: Total number of workers
# B08301_002E: Number of workers who work from home
variables = ['B08301_001E', 'B08301_002E']

# Years we are looking at
years = [2020, 2021, 2022]

# Create an empty list to hold the data
all_data = []

# Loop through each year 
for year in years:
    data = c.acs5.get(variables, geo={"for": "state:*"}, year=year)
    df = pd.DataFrame(data[1:], columns=data[0])
    df['year'] = year
    all_data.append(df)

# Combine all data into a single DataFrame
final_data = pd.concat(all_data, ignore_index=True)

# FIPS code to state name mapping
state_fips_to_name = {
    '01': 'Alabama', '02': 'Alaska', '04': 'Arizona', '05': 'Arkansas', '06': 'California',
    '08': 'Colorado', '09': 'Connecticut', '10': 'Delaware', '11': 'District of Columbia',
    '12': 'Florida', '13': 'Georgia', '15': 'Hawaii', '16': 'Idaho', '17': 'Illinois',
    '18': 'Indiana', '19': 'Iowa', '20': 'Kansas', '21': 'Kentucky', '22': 'Louisiana',
    '23': 'Maine', '24': 'Maryland', '25': 'Massachusetts', '26': 'Michigan', '27': 'Minnesota',
    '28': 'Mississippi', '29': 'Missouri', '30': 'Montana', '31': 'Nebraska', '32': 'Nevada',
    '33': 'New Hampshire', '34': 'New Jersey', '35': 'New Mexico', '36': 'New York', '37': 'North Carolina',
    '38': 'North Dakota', '39': 'Ohio', '40': 'Oklahoma', '41': 'Oregon', '42': 'Pennsylvania',
    '44': 'Rhode Island', '45': 'South Carolina', '46': 'South Dakota', '47': 'Tennessee', '48': 'Texas',
    '49': 'Utah', '50': 'Vermont', '51': 'Virginia', '53': 'Washington', '54': 'West Virginia',
    '55': 'Wisconsin', '56': 'Wyoming', '72': 'Puerto Rico'
}

# Map state FIPS codes to state names
final_data['state_name'] = final_data['state'].map(state_fips_to_name)

# Select relevant columns
final_data = final_data[['state_name', 'year', 'B08301_001E', 'B08301_002E']]

# Rename columns for clarity
final_data.columns = ['State Name', 'Year', 'Total Workers', 'Workers Who Work From Home']

# Handle missing data by replacing empty strings and None with NaN
final_data.replace({"": None, None: pd.NA}, inplace=True)

# Fill NaN values in the columns before pivoting
final_data['Total Workers'] = final_data['Total Workers'].fillna(0)  # Fill NaN in Total Workers with 0
final_data['Workers Who Work From Home'] = final_data['Workers Who Work From Home'].fillna("")  # Fill NaN in Workers Who Work From Home with an empty string

# Pivot the data to create separate columns for each year
pivoted_data = final_data.pivot(index='State Name', columns='Year', values=['Total Workers', 'Workers Who Work From Home'])

# Flatten MultiIndex columns
pivoted_data.columns = [f'{col[0]} - {col[1]}' for col in pivoted_data.columns]

# Reset index to bring 'State Name' back as a regular column
pivoted_data.reset_index(inplace=True)

# Final fillna to replace any remaining NaN values
pivoted_data = pivoted_data.fillna({"Total Workers - 2020": 0, "Total Workers - 2021": 0, "Total Workers - 2022": 0,
                                    "Workers Who Work From Home - 2020": "", "Workers Who Work From Home - 2021": "", "Workers Who Work From Home - 2022": ""})

# Format the 'Total Workers' and 'Workers Who Work From Home' columns with commas
for column in pivoted_data.columns:
    if 'Total Workers' in column or 'Workers Who Work From Home' in column:
        # Use pd.to_numeric to handle empty strings and non-numeric values, convert them to NaN if needed
        pivoted_data[column] = pd.to_numeric(pivoted_data[column], errors='coerce')
        # Format the numbers, replace NaNs with empty strings or another placeholder if needed
        pivoted_data[column] = pivoted_data[column].apply(
            lambda x: "{:,}".format(int(x)) if pd.notnull(x) else "0"  # You can replace "0" with "" 
        )

# Display the reshaped DataFrame
pivoted_data


,State Name,Total Workers - 2020,Total Workers - 2021,Total Workers - 2022,Workers Who Work From Home - 2020,Workers Who Work From Home - 2021,Workers Who Work From Home - 2022
0,Alabama,"2,095,195",0,0,"1,953,038",0,0
1,Alaska,"351,387","349,920","351,067","281,851","277,640","274,263"
2,Arizona,"3,179,988","3,174,911","3,244,419","2,709,243","2,628,121","2,610,395"
3,Arkansas,"1,294,646","1,295,639","1,304,084","1,198,182","1,184,798","1,177,402"
4,California,"18,239,892","18,283,118","18,353,469","14,963,132","14,580,124","14,313,290"
5,Colorado,"2,919,907","2,959,315","3,006,848","2,378,033","2,331,860","2,304,559"
6,Connecticut,"1,777,570","1,791,786","1,798,193","1,494,737","1,464,741","1,435,163"
7,Delaware,"451,927","459,571","468,881","394,953","389,829","389,861"
8,District of Columbia,"379,090","371,014","366,965","140,488","131,329","125,362"
9,Florida,"9,559,753","9,698,180","9,897,435","8,309,690","8,251,975","8,224,378"
